<a href="https://colab.research.google.com/github/neuromorph/TweetEval-Sentiment-HF/blob/main/TweetEval_Sentiment_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transformers, datasets

In [ ]:
from datasets import load_datasets

In [ ]:
senti_ds = load_dataset("tweet_eval", "sentiment")
senti_ds

In [ ]:
train_ds = senti_ds["train"]
train_ds

In [ ]:
train_ds[0], len(train_ds)

In [ ]:
senti_ds.set_format(type="pandas")
df = senti_ds["train"][:]
df.head()

In [ ]:
def label_int2str(row):
    return offensive["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

In [ ]:

df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)
df.boxplot("Words Per Tweet", by="label_name", grid=False,
          showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
    
print(tokenize(senti_ds["train"][:2]))

In [ ]:
senti_tok = senti_ds.map(tokenize, batched=True, batch_size=None)
print(senti_tok["train"].column_names)

In [ ]:

from transformers import AutoModelForSequenceClassification

num_labels = 3[]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

In [ ]:

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(senti_tok["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-tweet_eval-sentiment"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level="error")

In [ ]:
# from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=senti_tok["train"],
                  eval_dataset=senti_tok["validation"],
                  tokenizer=tokenizer)
trainer.train();

In [ ]:

preds_output = trainer.predict(senti_tok["validation"])
preds_output.metrics